## Module 6 : BioPython

As mentioned in the introduction, Biopython is a set of libraries to provide the ability to deal with “things” of interest to biologists working on the computer. In general this means that you will need to have at least some programming experience (in Python, of course!) or at least an interest in learning to program. Biopython’s job is to make your job easier as a programmer by supplying reusable libraries so that you can focus on answering your specific question of interest, instead of focusing on the internals of parsing a particular file format (of course, if you want to help by writing a parser that doesn’t exist and contributing it to Biopython, please go ahead!). So Biopython’s job is to make you happy!

One thing to note about Biopython is that it often provides multiple ways of “doing the same thing.” Things have improved in recent releases, but this can still be frustrating as in Python there should ideally be one right way to do something. However, this can also be a real benefit because it gives you lots of flexibility and control over the libraries. The tutorial helps to show you the common or easy ways to do things so that you can just make things work. To learn more about the alternative possibilities, look in the Cookbook, the Advanced section, the built in “docstrings” (via the Python help command, or the API documentation) or ultimately the code itself.

## Working with sequences

We’ll start with a quick introduction to the Biopython mechanisms for dealing with sequences, the Seq object, which we’ll discuss in more detail later.

Most of the time when we think about sequences we have in my mind a string of letters like ‘AGTACACTGGT’. You can create such Seq object with this sequence as follows:

In [ ]:
pip install biopython
pip install --upgrade biopython

In [ ]:
from Bio.Seq import Seq
my_seq = Seq("AGTACACTGGT")
my_seq

In [ ]:
my_string = "AGTACACTGGT"

In [ ]:
type(my_string)

In [ ]:
print(my_seq)

The Seq object differs from the Python string in the methods it supports. You can’t do this with a plain string:

In [ ]:
my_seq.complement()

In [ ]:
my_seq.reverse_complement()

In [ ]:
my_string.complement()

In [ ]:
my_seq.

The next most important class is the **SeqRecord** or Sequence Record. This holds a sequence (as a **Seq** object) with additional annotation including an identifier, name and description. The **Bio.SeqIO** module for reading and writing sequence file formats works with SeqRecord objects, which will be introduced below and covered in more detail later.

This covers the basic features and uses of the Biopython sequence class. Now that you’ve got some idea of what it is like to interact with the Biopython libraries, it’s time to delve into the fun, fun world of dealing with biological file formats!

## A usage example

Before we jump right into parsers and everything else to do with Biopython, let’s set up an example to motivate everything we do and make life more interesting. After all, if there wasn’t any biology in this tutorial, why would you want you read it?

Since I love plants, I think we’re just going to have to have a plant based example (sorry to all the fans of other organisms out there!). Having just completed a recent trip to our local greenhouse, we’ve suddenly developed an incredible obsession with Lady Slipper Orchids.

Of course, orchids are not only beautiful to look at, they are also extremely interesting for people studying evolution and systematics. So let’s suppose we’re thinking about writing a funding proposal to do a molecular study of Lady Slipper evolution, and would like to see what kind of research has already been done and how we can add to that.

After a little bit of reading up we discover that the Lady Slipper Orchids are in the Orchidaceae family and the Cypripedioideae sub-family and are made up of 5 genera: _Cypripedium_, _Paphiopedilum_, _Phragmipedium_, _Selenipedium_ and _Mexipedium_.

That gives us enough to get started delving for more information. So, let’s look at how the Biopython tools can help us. We’ll start with sequence parsing, but the orchids will be back later on as well - for example we’ll search PubMed for papers about orchids and extract sequence data from GenBank, extract data from Swiss-Prot from certain orchid proteins and work with ClustalW multiple sequence alignments of orchid proteins.

## Parsing sequence file formats

A large part of much bioinformatics work involves dealing with the many types of file formats designed to hold biological data. These files are loaded with interesting biological data, and a special challenge is parsing these files into a format so that you can manipulate them with some kind of programming language. However the task of parsing these files can be frustrated by the fact that the formats can change quite regularly, and that formats may contain small subtleties which can break even the most well designed parsers.

We are now going to briefly introduce the **Bio.SeqIO** module – you can find out later. We’ll start with an online search for our friends, the lady slipper orchids. To keep this introduction simple, we’re just using the NCBI website by hand. Let’s just take a look through the nucleotide databases at NCBI, using an Entrez online search (http://www.ncbi.nlm.nih.gov:80/entrez/query.fcgi?db=Nucleotide) for everything mentioning the text _Cypripedioideae_ (this is the subfamily of lady slipper orchids).

When this tutorial was originally written, this search gave us only 94 hits, which we saved as a FASTA formatted text file and as a GenBank formatted text file (files ls_orchid.fasta and ls_orchid.gbk, also included with the Biopython source code under docs/tutorial/examples/).

If you run the search today, you’ll get hundreds of results! When following the tutorial, if you want to see the same list of genes, just download the two files above or copy them from docs/examples/ in the Biopython source code. Below we will look at how to do a search like this from within Python.

### Simple FASTA parsing example

If you open the lady slipper orchids FASTA file <a href="../data/ls_orchid.fasta">ls_orchid.fasta</a> in your favourite text editor, you’ll see that the file starts like this:

> \>gi|2765658|emb|Z78533.1|CIZ78533 C.irapeanum 5.8S rRNA gene and ITS1 and ITS2 DNA
CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGGAATAAACGATCGAGTG
AATCCGGAGGACCGGTGTACTCAGCTCACCGGGGGCATTGCTCCCGTGGTGACCCTGATTTGTTGTTGGG
...

It contains 94 records, each has a line starting with “>” (greater-than symbol) followed by the sequence on one or more lines. Now try this in Python (printing the first 5 records):

In [ ]:
from Bio import SeqIO
for seq_record in list(SeqIO.parse("ls_orchid.fasta", "fasta"))[:5]:
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))

### Simple GenBank parsing example

Now let’s load the GenBank file <a href="../data/ls_orchid.gbk">ls_orchid.gbk</a> instead - notice that the code to do this is almost identical to the snippet used above for the FASTA file - the only difference is we change the filename and the format string:

In [ ]:
from Bio import SeqIO
for seq_record in list(SeqIO.parse("ls_orchid.gbk", "genbank"))[:5]:
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))

In [ ]:
help(SeqIO)

You’ll also notice that a shorter string has been used as the **seq_record.id** in this case.

## Connecting with biological databases

One of the very common things that you need to do in bioinformatics is extract information from biological databases. It can be quite tedious to access these databases manually, especially if you have a lot of repetitive work to do. Biopython attempts to save you time and energy by making some on-line databases available from Python scripts. Currently, Biopython has code to extract information from the following databases:

- Entrez (and PubMed) from the NCBI
- ExPASy
- SCOP

The code in these modules basically makes it easy to write Python code that interact with the CGI scripts on these pages, so that you can get results in an easy to deal with format. In some cases, the results can be tightly integrated with the Biopython parsers to make it even easier to extract information.

## What to do next

In [ ]:
from Bio import Blast
Blast.tool
'biopython'
Blast.email = ".....@gmail.com"

In [ ]:
from Bio import Blast
from Bio import SeqIO
record = SeqIO.read("m_cold.fasta", "fasta")
result_stream = Blast.qblast("blastn", "nt", record.seq)

***Saving BLAST results***

Whatever arguments you give the qblast() function, you should get back your results as a stream of bytes data (by default in XML format). The next step would be to parse the XML output into Python objects representing the search results (Section Parsing BLAST output), but you might want to save a local copy of the output file first. I find this especially useful when debugging my code that extracts info from the BLAST results (because re-running the online search is slow and wastes the NCBI computer time).

We need to be a bit careful since we can use result_stream.read() to read the BLAST output only once – calling result_stream.read() again returns an empty bytes object.

In [ ]:
with open("my_blast.xml", "wb") as out_stream:
    out_stream.write(result_stream.read())


result_stream.close()

After doing this, the results are in the file my_blast.xml and result_stream has had all its data extracted (so we closed it). However, the parse function of the BLAST parser (described in Parsing BLAST output) takes a file-like object, so we can just open the saved file for input as bytes:


In [ ]:
result_stream = open("my_blast.xml", "rb")

In [ ]:
from Bio import Blast
from Bio import SeqIO

# Define the path to your FASTA file
fasta_file = "ls_orchid.fasta"  # Replace with your actual file path

# Extract the first record
record = next(SeqIO.parse(fasta_file, "fasta"))
print(f"Using record: {record.id}")

# Run BLAST for the sequence
try:
    result_stream = Blast.qblast("blastn", "nt", record.seq)
    # Save or process the BLAST results as needed
    print(f"Successfully ran BLAST for record: {record.id}")
except Exception as e:
    print(f"An error occurred while processing the record: {e}")


In [ ]:
with open("my_blast.xml", "wb") as out_stream:
    out_stream.write(result_stream.read())


result_stream.close()
result_stream = open("my_blast.xml", "rb")